Reduccion Dimensionalidad PCA y LDA para Validation

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import IncrementalPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# 📌 Rutas
PREPROCESSED_PATH_TRAIN = r"C:\DAVID\CS\2025 0\machine_learning\Proyecto3\pequena_observacion_data\pre_processing"
PREPROCESSED_PATH_VALIDATION = r"C:\DAVID\CS\2025 0\machine_learning\P3_ML_GP6\pre_processing\validation"
OUTPUT_PATH = r"C:\DAVID\CS\2025 0\machine_learning\P3_ML_GP6\reduced_data"

# Crear carpeta si no existe
os.makedirs(OUTPUT_PATH, exist_ok=True)

# 📥 Inicializar IncrementalPCA (procesa por lotes)
pca = IncrementalPCA(n_components=50, batch_size=100)  # 🔹 Procesa 100 imágenes a la vez

# 📥 Entrenar PCA por lotes en `train`
print("⚙️ Entrenando IncrementalPCA por lotes con imágenes originales de TRAIN...")
archivos_train = sorted([f for f in os.listdir(PREPROCESSED_PATH_TRAIN) if f.endswith(".npy")])

for i in tqdm(range(0, len(archivos_train), 100)):  # 🔹 Procesa en lotes de 100 imágenes
    batch_files = archivos_train[i:i + 100]
    X_batch = [np.load(os.path.join(PREPROCESSED_PATH_TRAIN, file)).flatten() for file in batch_files]
    X_batch = np.vstack(X_batch)
    pca.partial_fit(X_batch)  # 🔹 Usa `partial_fit()` en lugar de `fit()`

# 💾 Guardar el modelo PCA entrenado
np.save(os.path.join(OUTPUT_PATH, "pca_components.npy"), pca.components_)
np.save(os.path.join(OUTPUT_PATH, "pca_mean.npy"), pca.mean_)

# 📥 Cargar y entrenar LDA con `train` (también por lotes)
lda = LDA(n_components=6)
X_train_pca = []

print("⚙️ Aplicando PCA a TRAIN para entrenar LDA...")
for i in tqdm(range(0, len(archivos_train), 100)):
    batch_files = archivos_train[i:i + 100]
    X_batch = [np.load(os.path.join(PREPROCESSED_PATH_TRAIN, file)).flatten() for file in batch_files]
    X_batch = np.vstack(X_batch)
    X_train_pca.append(pca.transform(X_batch))

X_train_pca = np.vstack(X_train_pca)
y_train = np.load(os.path.join(OUTPUT_PATH, "y_train.npy"), allow_pickle=True)
lda.fit(X_train_pca, y_train)

print("⚙️ Aplicando PCA y LDA a Validation...")

# 📂 Obtener la lista de imágenes preprocesadas de `validation`
archivos_val = sorted([f for f in os.listdir(PREPROCESSED_PATH_VALIDATION) if f.endswith(".npy")])

X_reduced_pca = []
X_reduced_lda = []

for file in tqdm(archivos_val):
    image_path = os.path.join(PREPROCESSED_PATH_VALIDATION, file)
    img = np.load(image_path).flatten()

    # Aplicar PCA correctamente
    img_pca = pca.transform(img.reshape(1, -1))
    X_reduced_pca.append(img_pca)

    # Aplicar LDA
    img_lda = lda.transform(img_pca)
    X_reduced_lda.append(img_lda)

# 📥 Convertir listas a arrays numpy
X_reduced_pca = np.vstack(X_reduced_pca)
X_reduced_lda = np.vstack(X_reduced_lda)

# 💾 Guardar los archivos reducidos
np.save(os.path.join(OUTPUT_PATH, "pca_reduced_validation.npy"), X_reduced_pca)
np.save(os.path.join(OUTPUT_PATH, "lda_reduced_validation.npy"), X_reduced_lda)

print("✅ Reducción de dimensionalidad para Validation completada.")


📥 Cargando imágenes originales de TRAIN para reentrenar PCA...


  0%|          | 0/10015 [00:00<?, ?it/s]

100%|██████████| 10015/10015 [05:52<00:00, 28.37it/s]


MemoryError: Unable to allocate 30.2 GiB for an array with shape (10015, 810000) and data type float32